# ENGRI 1120: Series Cell-free Production and Recovery of the mRNA BNT-162b2 Vaccine

#### Chike Iwuala, Osvaldo Hernandez, Jake Pena, Anthony Parlatore

<img src="figs/ProcessDesign3.png" style="width:50%">

## Introduction
The project introduction goes here. Typically 2-3 paragraphs.

__Suggestions__:
* Paragraph 1: Describe the motivation and background information on the product and the vaccine 
* Paragraph 2/3: Summary of your project. Describe the number of units (don't forget the heat exchangers), details about the units’ operation, and your project’s financials. 

You can replace the figure above with your system.

COVID-19 BNT-162b2 (tozinameran) vaccine is a mRNA molecule encapsulated in lipid nanoparticles (LNPs), which share many similarities to a traditional liposome. Once the LNP binds to a host cell, the LNPs transfers the mRNA, thereby enabling the expression of the SARS-CoV-2 spike protein and “achieving immunity” from the deadly SARS-CoV-2 infection. The tozinameran vaccine is administered in two doses. It was observed in a study — with about 43,548 participants — that the tozinameran vaccine had an efficacy of 95 percent after the second dose. For comparison, the Moderna vaccine has 94.1 percent after the second dose. This highlights the need to develop an economically feasible process that maximizes the production of the BNT-162b2 mRNA molecule so as to provide the vaccine at a lower cost, thereby increasing its potential availability to the public and reducing the competition for COVID-19 vaccines. This project will only lay out a process that produces the target amount of BNT-162b2 mRNA molecule. (It does not include the extra step of encasing it in an LNP.)

The manufacturing process of the BNT-162b2 mRNA target molecule will need two syringe pumps, one stream splitter, 15 reactor chips, three heat exchangers, 1 magical separator unit (MSU), and two storage vessels. One of the syringe pumps transports the BNT-162b2 gene into a stream splitter, which will ensure that each reactor chip is supplied with an appropriate amount necessary to produce the BNT-162b2 mRNA molecule, while the other provides a stock solution of PURExpress, which is responsible for the expression of the BNT-162b2 gene, to the first reactor chip. Each reactor chip has a limited volume (about 100 μL), which makes it impossible to use up all the starting materials and manage to produce 0.1 μmoles of the BNT-162b2 mRNA target molecule in one reactor. This fact explains the need to include multiple reactor chips, which will run in-series, in the design. After the BNT-162b2 mRNA molecule and other unreacted starting materials has exited the last reactor chip, they will go through the MSU that is 100 percent effective. The MSU will divide the mRNA target molecule and the unreacted starting materials into two streams. These streams will transfer the aforementioned product and unreacted starting materials  into 2 different storage vessels. 
It is also important to note that the units described above operate optimally at temperatures that are significantly greater than the maximum temperature of stability for the mRNA molecule (-25℃). This issue can be resolved by including heat exchangers in the design. The first heat exchanger will be placed behind the stream splitter (and in front of syringe pump 1) and the second will be placed in front of syringe pump 2. These heat exchangers will heat the PURExpress and BNT-162b2 gene solutions to the operating temperature of the reactor chips (37℃). The third heat exchanger, which will be stationed after the last reactor chip, cools the mRNA molecule below -25℃.

The total starting budget for this design process will be ***374,518,725 dollars***. This figure was derived from the assumption that we buy enough PURExpress to last in our manufacturing process for five years; the cost of PURExpress under this assumption is ***374,490,000 dollars***(about 99 percent of the total budget). The cost of the manufacturing units was estimated to be about ***28,725 dollars***. Under this design process, each micromole of mRNA-BNT162b2 would have to be sold at a retail price of ***1956.07 dollars*** to generate the same income as an alternative investment yielding 4.1 percent per year.

## Materials and Methods

### Project Setup

In [22]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Downloads/ChemEProject-main 3/Final Project`
  No Changes to `~/Downloads/ChemEProject-main 3/Final Project/Project.toml`
  No Changes to `~/Downloads/ChemEProject-main 3/Final Project/Manifest.toml`


In [23]:
# load reqd packages and set paths -
using JLD2
using FileIO
using PrettyTables
using DataFrames
using GLPK

# setup paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT, "data");

#### Load the project code library
The call to the `include` function loads the `ENGRI-1120-Project-CodeLib.jl` library into the notebook; the library contains functions we can use during the project. In particular, it includes the function:

* The `compute_optimal_extent(stoichiometric_matrix::Array{Float64,2}, default_bounds_array::Array{Float64,2},
    species_bounds_array::Array{Float64,2}, objective_coefficient_array::Array{Float64,1}; min_flag::Bool = true) -> Tuple` function calls the [GLPK](https://www.gnu.org/software/glpk/) linear program solver. The `results` tuple contains several things, but the important ones are `calculated_flux_array`, `objective_value`, and the status/exit flags `status_flag` and `exit_flag` (which let us know if the solver successfully found a solution).
* The `build(model::Type{MSULatticeModel}; ṅₒ::Float64, L::Int64, u::Float64, d::Float64) -> MSULatticeModel` function builds a [Binary tree](https://en.wikipedia.org/wiki/Binary_tree) of Magical Separation Units (MSUs). Arguments: $ṅₒ$ denotes the species mole flow rate into the separation system, $L$ denotes the number of layers of the tree, $u$ denotes the `up` factor (split for the `up` path), and $d$ denotes the `down` factor (the split for the `down` path). This function returns the `MSULatticeModel` model, which contains the column array `data` holding the species mole flow rate for each node in the tree. 
* The `build_nodes_dictionary(levels::Int64) -> Dict{Int64,Array{Int64,1}}` function constructs a dictionary of node indexes for each level of the tree; keys are the tree levels.

In [24]:
include("ENGRI-1120-Project-CodeLib.jl");

In [25]:
# load the model file -
model = load(joinpath(_PATH_TO_DATA, "ENGRI-1120-BNT162b2-Model.jld2"))["model"]

┌ Warning: the following fields are present in type DataFrame saved in the file but not present in the type the workspace:
│ 
│ metadata
│ colmetadata
│ allnotemetadata,
│ 
│ Data in these fields will not be accessible
└ @ JLD2 /Users/osvaldohernandez/.julia/packages/JLD2/k9Gt0/src/data/reconstructing_datatypes.jl:196


Dict{String, Any} with 7 entries:
  "stochiometric_matrix" => [-1.0 0.0 … 0.0 -1.0; -1.0 0.0 … -1.0 0.0; … ; 1.0 …
  "list_of_reactions"    => ["TX_BNT_162b2_binding", "TX_BNT_162b2_open", "BNT_…
  "reaction_table"       => 6×7 DataFrame…
  "flux_bounds_array"    => [-1000.0 1000.0; 0.0 1000.0; … ; 0.0 1000.0; 0.0 10…
  "mRNA_sequence"        => ['C', 'U', 'C', 'U', 'U', 'A', 'U', 'U', 'U', 'G'  …
  "list_of_species"      => ["G_BNT_162b2", "T7RNAP", "M_atp_c", "M_utp_c", "M_…
  "gene_sequence"        => ['G', 'A', 'G', 'A', 'A', 'T', 'A', 'A', 'A', 'C'  …

In [26]:
# get stuff from the model data structure -
S = model["stochiometric_matrix"]; # fix the spelling in the model file
flux_bounds_array = model["flux_bounds_array"];
list_of_species = model["list_of_species"];
list_of_reactions = model["list_of_reactions"];
reaction_table = model["reaction_table"];
gene_sequence = model["gene_sequence"];

In [27]:
reaction_table

,id,forward
,String,String
1,TX_BNT_162b2_binding,G_BNT_162b2+T7RNAP
2,TX_BNT_162b2_open,G_BNT_162b2_T7RNAP_closed
3,BNT_162b2_transcription,G_BNT_162b2_T7RNAP_open+798*M_atp_c+1004*M_utp_c+1060*M_ctp_c+1312*M_gtp_c
4,mRNA_BNT_162b2_degradation,mRNA_BNT_162b2
5,RNAP_deactivation,T7RNAP
6,GENE_deactivation,G_BNT_162b2


In [28]:
# How many species and reactions?
(ℳ, ℛ) = size(S);

In [29]:
# initialize -
species_index_table_data = Array{Any,2}(undef, ℳ, 2);

# build table -
for i ∈ 1:ℳ
    species_index_table_data[i,1] = i;
    species_index_table_data[i,2] = list_of_species[i];
end

# setup header -
species_index_header_table = (["Index", "Species"]);

# build table -
pretty_table(species_index_table_data; header=species_index_header_table);

┌───────┬───────────────────────────┐
│ Index │                   Species │
├───────┼───────────────────────────┤
│     1 │               G_BNT_162b2 │
│     2 │                    T7RNAP │
│     3 │                   M_atp_c │
│     4 │                   M_utp_c │
│     5 │                   M_ctp_c │
│     6 │                   M_gtp_c │
│     7 │            mRNA_BNT_162b2 │
│     8 │                   M_ppi_c │
│     9 │                   M_amp_c │
│    10 │                   M_ump_c │
│    11 │                   M_cmp_c │
│    12 │                   M_gmp_c │
│    13 │           T7RNAP_inactive │
│    14 │      G_BNT_162b2_inactive │
│    15 │ G_BNT_162b2_T7RNAP_closed │
│    16 │   G_BNT_162b2_T7RNAP_open │
└───────┴───────────────────────────┘


#### Setup the constants, feed rates and compositions

In [30]:
# how many chips in series?
number_of_chips = 15;

# what fraction of the mRNA degrades?
δ = 0.10;

# MSU split ratio -
θ = 1.0;

# Setup constants for transcription -
L = length(gene_sequence);
K = 0.116; # saturation constant units: μmol/L; Source: ACS Synth. Biol. 2018, 7, 8, 1844–1857 https://doi.org/10.1021/acssynbio.7b00465
v̇ₜ = (90.0)*(60); # units: nt/s; Source: BIND: 111871
u = 0.95; # u-factor; Source: ACS Synth. Biol. 2018, 7, 8, 1844–1857 https://doi.org/10.1021/acssynbio.7b00465

# volume -
V = 100.0*(1/1e6); # liquid reaction volume on each chip units: L

# Stock solution: PURExpress -> fed into chips by pump 2 (stream 2)
# PURExpress flows into the chip in stream 2
T7RNAP = 100.0;          # concentration in PURExpress units: μmol/L
M_atp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_utp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_ctp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_gtp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L

# Stock solution: DNA -> feed into splitter by pump 1 and then into chip (always stream 1)
G_BNT_162b2 = 1.0;     # gene concentration in stock solution units: μmol/L

# Volumetric flow rates from the pump *into* the splitter unit or chip - our base case will 1 ml/min; thus, let's
# scale by the number of chips
Ḟ₁ = number_of_chips*1000.0*(1/1e6); # volumetric flow rate of syringe pump 1 units: L/min
Ḟ₂ = number_of_chips*1000.0*(1/1e6); # volumetric flow rate of syringe pump 2 units: L/min

# stuff for the tables -
current_table_counter = 0; # do not change me

#### Specify the inputs streams

##### a) Specify the composition of feed stream 1
By default, this stream contains only the gene encoding the mRNA BNT-162b2 product. The mainstream is split into sub-streams that are fed into each chip.

In [31]:
# setup feed compostions for feed stream 1 
ṅ₁ = zeros(ℳ); # default is zero, correct specific values -
ṅ₁[1] = G_BNT_162b2*Ḟ₁*(1/number_of_chips); # units: μmol/min

##### b) Specify the composition of feed stream 2
By default, feed stream 2 contains the [PURExpress](https://www.neb.com/products/e6800-purexpress-invitro-protein-synthesis-kit#Product%20Information), which has everything we need to make our mRNA product of interest _expect_ the linear DNA. 

In [32]:
# setup feed compostions for feed stream 2: this feed goes into chip 1
ṅ₂ = zeros(ℳ); # default is zero, then correct specific values -
ṅ₂[2] = T7RNAP*Ḟ₂;  # units: μmol/min
ṅ₂[3] = M_atp_c*Ḟ₂; # units: μmol/min
ṅ₂[4] = M_utp_c*Ḟ₂; # units: μmol/min
ṅ₂[5] = M_ctp_c*Ḟ₂; # units: μmol/min
ṅ₂[6] = M_gtp_c*Ḟ₂; # units: μmol/min

### Flux balance analysis setup
* Describe the flux balance analysis method. 
* How did your group update the default species and flux bounds to model ths process?
* What objective function did you choose?

Flux balance analysis is a mathematical approach that is used to analyze the flow of metabolites through a metabolic network. This makes it possible to predict the growth rate of an organism or the rate of production of an important metabolite, which in this particular design would be mRNA BNT-162b2 target molecule. Flux balance analysis is only functional ***if the streams that are involved operate at steady state, so this assumption must be noted***.
This specific FBA utilizes a mole-based approach to compute for a combination of optimal reaction extents that satisfies the linear objective, which is to maximize the transcription of the BNT-162b2 gene.

The first and most important step in an FBA problem is to list the constraints. The first is rooted in the stoichiometries of the metabolic reactions that play a role in the production of the mRNA BNT-162b2 molecule. This can be resolved by creating a numerical matrix that contains the stoichiometric coefficients of each species in all reactions involved in the production of the mRNA BNT-162b2 molecule. Another important constraint is that there cannot be negative flow rates for any species in any reaction that result in the production of the mRNA. The final constraint is that all reaction extents must have a lower and upper bound. 

The above statements for each reactor chip can be summarized by the following formulas: 

The steady-state species mole balances are given by the equation:

$$\sum_{s∈S^{+}}{\dot{n}_{si}}-\sum_{k∈S^{-}}{\dot{n}_{ki}}+\sum_{j∈R}{\sigma_{ij}\dot{\epsilon}_{j}=0}, \quad \forall i∈M$$

From the second paragraph, $\dot{n}_{i,j} \geq 0$ for all i and j. Then, the (unknown) open extents $\dot{\epsilon}_{j}$ are the solution of a linear programming problem in which the linear objective ***N***, which is to maximize the production of mRNA-BNT-162b2.

$$maximize \, N = \sum_{j∈R}{c_{j}\dot{\epsilon}_{j}}$$ and is subject to the following linear constraints:
$$\sum_{j∈R}{\sigma_{ij}\dot{\epsilon}_{j}} \geq -\sum_{s∈S^{+}}{\dot{n}_{si}} , \quad \forall i∈M $$
$$\sum_{k∈S^{-}}{\dot{n}_{ki}} \geq 0, \quad \forall i∈M $$
$$L_{j}\le \dot{\epsilon}_{j} \le U_{j} , \quad \forall j∈R$$

The lower and upper bounds for the species and reactions involved in the production of mRNA-BNT-162b2 are provided in the tables below.  

List of Terms
* The term M = set of species involved in the production of mRNA
* The term R = set of reactions involved in the production of mRNA
* The term $s∈S^{+}$ = the set of streams that enter the reactor chip 
* The term $s∈S^{-}$ = streams the set of streams that leave the reactor chip 
* The term $\dot{n}_{s,i}$ = the mole flow rate of species ***i*** in input stream ***s*** (units: micromoles/min) involved in the production of mRNA
* The term $\dot{n}_{k,i}$ = the mole flow rate of species ***i*** in exit stream ***k*** (units: micromoles/min) involved in the production of mRNA
* The term $\sigma_{i,j}$ = stoichiometric coefficient of species ***i*** in reaction ***j*** involved in the production of mRNA
* The term $\dot{\epsilon}_{j}$ = open extent of reaction (units: micromoles/min)
* The term L = lower bound for reaction j involved in the production of mRNA
* The term U = upper bound for reaction j involved in the production of mRNA

#### Species bounds

In [33]:
# species bounds array -
species_bounds_array = [-(ṅ₁ .+ ṅ₂) 10000.0*ones(ℳ,1)];

#### Reaction bounds
In the series case, reaction bounds will change for every chip. Let's set a default upper bound and then update the other bounds below. If you are a brave soul, you can alter the per-chip bounds, but you don't need to do so.

In [34]:
# initialize -
flux_bounds_array = zeros(ℛ,2);
flux_bounds_array[:,2] .= 1000.0; # large default upper bound

In [35]:
# initialize -
reaction_index_table_data = Array{Any,2}(undef, ℛ, 4);

# build table -
for i ∈ 1:ℛ
    reaction_index_table_data[i,1] = i;
    reaction_index_table_data[i,2] = list_of_reactions[i];
    reaction_index_table_data[i,3] = flux_bounds_array[i,1];
    reaction_index_table_data[i,4] = flux_bounds_array[i,2];
end

# setup title string -
reaction_table_title = "Table $(current_table_counter+=1): Reaction index-name mapping table."

# setup header -
reaction_index_header_table = (["Index", "Reaction", "lower bound", "upper bound"], ["", "", "μmol/min", "μmol/min"]);

# build table -
pretty_table(reaction_index_table_data, title=reaction_table_title; header=reaction_index_header_table);

Table 1: Reaction index-name mapping table.
┌───────┬────────────────────────────┬─────────────┬─────────────┐
│ Index │                   Reaction │ lower bound │ upper bound │
│       │                            │    μmol/min │    μmol/min │
├───────┼────────────────────────────┼─────────────┼─────────────┤
│     1 │       TX_BNT_162b2_binding │         0.0 │      1000.0 │
│     2 │          TX_BNT_162b2_open │         0.0 │      1000.0 │
│     3 │    BNT_162b2_transcription │         0.0 │      1000.0 │
│     4 │ mRNA_BNT_162b2_degradation │         0.0 │      1000.0 │
│     5 │          RNAP_deactivation │         0.0 │      1000.0 │
│     6 │          GENE_deactivation │         0.0 │      1000.0 │
└───────┴────────────────────────────┴─────────────┴─────────────┘


#### Objective coefficient array
* What is the objective you used?

The objective is to maximize BNT-162b2 transcription, which is why the objective vector is -1 for index 3.

In [36]:
# setup the objective coefficient array -
obj_vector = zeros(ℛ);
obj_vector[3] = -1; # why negative?

## Results and Discussion 

### Compute the optimal extent of reaction and exit stream for first chip

In [37]:
# initialize some space to store tmp soln -
tmp_sim_storage_space = Dict{Int64,Tuple{Array{Float64,1},Array{Float64,1}}}();

# compute -
ṅ₃ = zeros(ℳ);
for chip_index ∈ 1:number_of_chips
    
    if (chip_index == 1)
        ṅ_chip = ṅ₂
    else
        ṅ_chip = ṅ₃
    end
        
    # update the bounds model -
    F̂₁ = ((chip_index/number_of_chips)*Ḟ₁)/(Ḟ₂+(chip_index/number_of_chips)*Ḟ₁)
    F̂₂ = Ḟ₂/(Ḟ₂+(chip_index/number_of_chips)*Ḟ₁);
    
    # Estimate the RNAP and GENE concentration -
    Rₜ = T7RNAP*F̂₂;                          # effective RNAP concentratation on the chip for the bounds units: μmol/L
    GENE = G_BNT_162b2*F̂₁ + G_BNT_162b2*F̂₂  # effective GENE concentratation on the chip for the bounds units: μmol/L
    flux_bounds_array[3,:] .= Rₜ*(v̇ₜ/L)*u*(GENE/(K+GENE))*V; # equality constraint

    # Setup bound for degradation (lower bound)
    flux_bounds_array[4,1] = δ*flux_bounds_array[3,1];
    
    # compute the optimal flux, and then estimate the output on the chip
    result = compute_optimal_extent(S, flux_bounds_array, species_bounds_array, obj_vector);
    ϵ̇ = result.calculated_flux_array;
    ṅ₃ = (ṅ₁ + ṅ_chip) + S*ϵ̇;   # compute the output from the chip -
    
    # now, the outlet from chip i-1 is the input to chip 1 (in stream 2), but stream 1 stays the same
    species_bounds_array = [-(ṅ₁ .+ ṅ_chip) 10000.0*ones(ℳ,1)];
    
    # grab -
    tmp_sim_storage_space[chip_index] = (ṅ₃,ϵ̇);
end

In [38]:
system_flow_table_data = Array{Any,2}(undef, ℳ, number_of_chips+2);

# populate the table -
for i ∈ 1:ℳ
    system_flow_table_data[i,1] = list_of_species[i];
    system_flow_table_data[i,2] = i;
    
    for chip_index ∈ 1:number_of_chips
        ṅ₃ = tmp_sim_storage_space[chip_index][1];
        system_flow_table_data[i,chip_index+2] = round(ṅ₃[i], digits=3);
    end
end

# build header -
# names 
flow_table_name_row = Array{String,1}()
push!(flow_table_name_row, "Species");
push!(flow_table_name_row, "index i")
for chip_index ∈ 1:number_of_chips
    push!(flow_table_name_row, "Chip $(chip_index)")
end

# units
flow_table_units_row = Array{String,1}()
push!(flow_table_units_row, "");
push!(flow_table_units_row, "")
for chip_index ∈ 1:number_of_chips
    push!(flow_table_units_row, "μmol/min")
end


# title -
flow_table_title = "Table $(current_table_counter+=1): Chip species mole flow rates table"

# header -
flux_table_header = (flow_table_name_row, flow_table_units_row)

# show -
pretty_table(system_flow_table_data, title=flow_table_title; header=flux_table_header)

Table 2: Chip species mole flow rates table
┌───────────────────────────┬─────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│                   Species │ index i │   Chip 1 │   Chip 2 │   Chip 3 │   Chip 4 │   Chip 5 │   Chip 6 │   Chip 7 │   Chip 8 │   Chip 9 │  Chip 10 │  Chip 11 │  Chip 12 │  Chip 13 │  Chip 14 │  Chip 15 │
│                           │         │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │
├───────────────────────────┼─────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│               G_BNT_162b2 │       1 │    0.001 │    0.002 │    0.003 │    0.004 │    0.005 │    0.006 │    0.007 │    0.008 │    0.009

In [39]:
# this will print the last few chips (in case the table formatting gets nutty)
system_flow_table_data

16×17 Matrix{Any}:
 "G_BNT_162b2"                 1     0.001  …     0.013     0.014     0.015
 "T7RNAP"                      2     1.5          1.5       1.5       1.5
 "M_atp_c"                     3  1491.76      1419.75   1415.2    1410.81
 "M_utp_c"                     4  1489.64      1399.03   1393.31   1387.78
 "M_ctp_c"                     5  1489.06      1393.4    1387.36   1381.53
 "M_gtp_c"                     6  1486.46   …  1368.05   1360.58   1353.36
 "mRNA_BNT_162b2"              7     0.009        0.091     0.096     0.101
 "M_ppi_c"                     8    43.084      419.775   443.546   466.524
 "M_amp_c"                     9     0.824        8.025     8.48      8.919
 "M_ump_c"                    10     1.036       10.097    10.669    11.222
 "M_cmp_c"                    11     1.094  …    10.66     11.264    11.848
 "M_gmp_c"                    12     1.354       13.195    13.942    14.664
 "T7RNAP_inactive"            13     0.0          0.0       0.0       0.0
 

### Design a downstream seperation using Magical Seperator Units (MSU)

In [40]:
# default -
ṅ₃ = tmp_sim_storage_space[number_of_chips][1]

# build a downstream seperation process with this number of levels:
number_of_levels = 2; # includes zero

In [41]:
# initialize -
tmp_storage_dict = Dict{Int64, MSULatticeModel}();

# is_product_vector -
is_product_vector = zeros(ℳ);
is_product_vector[7] = 1;

# compute the composition array -
for i ∈ 1:ℳ

    if (is_product_vector[i] == 1)
        msu_lattice_model = build(MSULatticeModel; ṅₒ = ṅ₃[i], L = number_of_levels , u = θ, d = (1 - θ));
    else
        msu_lattice_model = build(MSULatticeModel; ṅₒ = ṅ₃[i], L = number_of_levels , u = (1 - θ), d = θ);
    end
    
    # grab -
    tmp_storage_dict[i] = msu_lattice_model;
end

# grab the leaves -
nodes_dict = build_nodes_dictionary(number_of_levels);
children_dict = build_children_dictionary(nodes_dict);
tree_leaves = nodes_dict[number_of_levels-1];

# build a composition array -
number_of_nodes = length(tree_leaves);
composition_array = Array{Float64,2}(undef, number_of_nodes, ℳ);
for i ∈ 1:ℳ
    data = tmp_storage_dict[i].data;
    for j ∈ 1:number_of_nodes
        composition_array[j,i] = data[tree_leaves[j]]
    end
end

# make a pretty table and show the leaves of the tree -

# initialize -
sep_tree_flow_table_data = Array{Any,2}(undef, ℳ, length(tree_leaves) + 3)
for i ∈ 1:ℳ
    sep_tree_flow_table_data[i,1] = list_of_species[i];
    sep_tree_flow_table_data[i,2] = i;
    sep_tree_flow_table_data[i,3] = ṅ₃[i] # put node 0 in table -
        
    for j ∈ 1:length(tree_leaves)
        sep_tree_flow_table_data[i,3+j] = composition_array[j,i]
    end
end

# labels row -
label_row = Array{String,1}();
push!(label_row,"Species");
push!(label_row,"index i")
push!(label_row,"N0")
for j ∈ 1:length(tree_leaves)
    push!(label_row, "N$(tree_leaves[j])");
end

# units row -
units_row = Array{String,1}();
push!(units_row, "");
push!(units_row, "");
for j ∈ 1:length(tree_leaves)+1
    push!(units_row, "μmol/min");
end

# header -
sep_tree_flow_table_header = (label_row, units_row);

# set title -
title = "Table $(current_table_counter+=1): Magical Seperator Unit (MSU) flow table; N0 denotes the feed while N⋆ denotes the leaves of the tree."

# show -
pretty_table(sep_tree_flow_table_data, title=title; header=sep_tree_flow_table_header)

Table 3: Magical Seperator Unit (MSU) flow table; N0 denotes the feed while N⋆ denotes the leaves of the tree.
┌───────────────────────────┬─────────┬──────────┬──────────┬──────────┐
│                   Species │ index i │       N0 │       N2 │       N3 │
│                           │         │ μmol/min │ μmol/min │ μmol/min │
├───────────────────────────┼─────────┼──────────┼──────────┼──────────┤
│               G_BNT_162b2 │       1 │    0.015 │      0.0 │    0.015 │
│                    T7RNAP │       2 │      1.5 │      0.0 │      1.5 │
│                   M_atp_c │       3 │  1410.81 │      0.0 │  1410.81 │
│                   M_utp_c │       4 │  1387.78 │      0.0 │  1387.78 │
│                   M_ctp_c │       5 │  1381.52 │      0.0 │  1381.52 │
│                   M_gtp_c │       6 │  1353.36 │      0.0 │  1353.36 │
│            mRNA_BNT_162b2 │       7 │ 0.100592 │ 0.100592 │      0.0 │
│                   M_ppi_c │       8 │  466.524 │      0.0 │  466.524 │
│            

In [42]:
# build mol frac composition table -

# construct mol frac array -
mol_frac_array = Array{Float64,2}(undef, ℳ, length(tree_leaves)+1);

# node 0 -
ṅ₃_total = sum(ṅ₃);
for i ∈ 1:ℳ
    mol_frac_array[i,1] = ṅ₃[i]*(1/ṅ₃_total);    
end

# get the sums along rows -
ṅ_total = sum(composition_array,dims = 2);
for node ∈ 1:length(tree_leaves)
    for i ∈ 1:ℳ
        mol_frac_array[i,node+1] = composition_array[node,i]*(1/ṅ_total[node]);
    end
end

# initialize -
sep_tree_mol_frac_table_data = Array{Any,2}(undef, ℳ, length(tree_leaves) + 3)
for i ∈ 1:ℳ
    sep_tree_mol_frac_table_data[i,1] = list_of_species[i];
    sep_tree_mol_frac_table_data[i,2] = i;
    sep_tree_mol_frac_table_data[i,3] = round(mol_frac_array[i,1], digits=4) # put node 0 in table -
        
    for j ∈ 1:length(tree_leaves)
        sep_tree_mol_frac_table_data[i, 3+j] = round(mol_frac_array[i,j+1], digits=4)
    end
end

# labels row -
label_mft_row = Array{String,1}();
push!(label_mft_row,"Species");
push!(label_mft_row,"index i")
push!(label_mft_row,"N0")
for j ∈ 1:length(tree_leaves)
    push!(label_mft_row, "N$(tree_leaves[j])");
end

# units row -
units_mft_row = Array{String,1}();
push!(units_mft_row, "");
push!(units_mft_row, "");
for j ∈ 1:length(tree_leaves)+1
    push!(units_mft_row, "mole frac");
end

# header -
sep_tree_mft_table_header = (label_mft_row, units_mft_row);

# set title -
title_mft = "Table $(current_table_counter+=1): Magical Seperator Unit (MSU) composition table."

# show -
pretty_table(sep_tree_mol_frac_table_data, title=title_mft; header=sep_tree_mft_table_header)

Table 4: Magical Seperator Unit (MSU) composition table.
┌───────────────────────────┬─────────┬───────────┬───────────┬───────────┐
│                   Species │ index i │        N0 │        N2 │        N3 │
│                           │         │ mole frac │ mole frac │ mole frac │
├───────────────────────────┼─────────┼───────────┼───────────┼───────────┤
│               G_BNT_162b2 │       1 │       0.0 │       0.0 │       0.0 │
│                    T7RNAP │       2 │    0.0002 │       0.0 │    0.0002 │
│                   M_atp_c │       3 │    0.2333 │       0.0 │    0.2333 │
│                   M_utp_c │       4 │    0.2295 │       0.0 │    0.2295 │
│                   M_ctp_c │       5 │    0.2284 │       0.0 │    0.2284 │
│                   M_gtp_c │       6 │    0.2238 │       0.0 │    0.2238 │
│            mRNA_BNT_162b2 │       7 │       0.0 │       1.0 │       0.0 │
│                   M_ppi_c │       8 │    0.0771 │       0.0 │    0.0771 │
│                   M_amp_c │  

### Financial analysis

* Compute the [Net Present Value](https://varnerlab.github.io/ENGRI-1120-IntroToChemE-Book/chapter-1-dir/money-balances.html#net-present-value-npv) of your design using the cost spreadsheet on canvas.
* What price for the mRNA product do we need to set to have a NPV equal to zero?
* How could you improve the financial performance/NPV of your design?

This calculation for the Net Present Value of the design depends on a few assumptions. They are: 
* The production of mRNA-BNT162b2 runs 24/7 for 5 straight years. This is only possible through the use of machines. 
* None of the units (i.e. heat exchangers, MSUs, streams, reactor chips) including the machines break down during those five years. 
* It is possible to buy enough PURExpress to last for 5 years for this process in the first year. This will constitute a huge part of the startup cost. 
* The PURExpress that is bought will not expire during those 5 years, i.e. its contents will always act in their ideal capabilities during those 5 years.
* We only sell the mRNA that is produced from this process. Unreacted starting materials (PURExpress and BNT-162b2 gene) are labelled as "waste," and are subsequently disposed.
* We are always able to sell the mRNA that is produced at a certain price — so no discounts — 24/7/365 during those five years. 
* The only cash flow after the first year comes from selling the mRNA product, and remains constant every year. 


The Net Present Value of this design can then be expressed as: 
$$ NPV = CF_1 + \sum_{t=2}^{T}D^{-1}_{t,1}(CF_t) = CF_1 + \sum_{t=2}^{T}(1+r)^{1-t}(CF_t)$$
where: 
* $CF_{1}$ = startup cost, i.e. PURExpress for 5 years and other components for manufacturing (unit: dollars)
* $D_{t,1}^{-1}$ = multi-period discount factor = $((1+r)^{t-1})^{-1} = (1+r)^{1-t}$
* $r$ = rate of return
* $CF_t$ = cash flow at time t (unit: dollars)

$\quad$

Since the cash flow after $t = 1$ is the income generated from ***only*** selling mRNA, and $r = 0.041$, the NPV can be expressed as: 
$$ NPV = CF_1 + \sum_{t=2}^{5}(1.041)^{1-t}(a \cdot N)$$
where:
* a = price per micromole of mRNA (units: dollars/μmol)
* N = amount of micromoles produced in a year (units: μmol)

$\quad$

First, determine $CF_1$. We start by calculating the number of PURExpress we need if this process were to run all days of the week for 5 years. 

$$Total\, PURExpress= 525600\, min \cdot \dot{F_{2}} \cdot 5\, years$$
where
* $\dot{F_{2}}$ = volumetric flow rate of syringe pump 2 (unit: L/min)

$\quad$

In this design, $\dot{F_{2}} = 0.015\,L/min$, so the total number of PURExpress that is consumed in 5 years is: 
$$Total\, PURExpress = 39420\,L$$

Additionally, $1L \, of\, PURExpress = 9500\, dollars$, so 
$$ Cost\, of\, PURExpress = -374,490,000 \, dollars $$

$\quad$ 

The cost of all manufacturing units used in this design is 28,725 dollars.

Adding all costs together, 
$$ CF_1 = -374,518,725 \, dollars $$

$\quad$

Next, we need to calculate ***N***, which is the number of micromoles of mRNA produced by this design in a year. 

$$ 
\begin{eqnarray}
N = (\dot{P}) \cdot (525600\, min)
\end{eqnarray}
$$
where: 
* $\dot{P}$ = flow rate of mRNA-BNT-162b2 in node 2 (μmol/min)
Our design process has a $\dot{P} = 0.100592$, therefore: 

$$ N = (0.100592\,{\mu mol\over min}) \cdot (525600\, min) = 52871.1552\, \mu mol$$

$\quad$

Finally, we can assemble a full expression for NPV. It is: 
$$ NPV = CF_1 + \sum_{t=2}^{5}(1.041)^{1-t}(a \cdot N) = -374518725 + \sum_{t=2}^{5}(1.041)^{1-t}(52871.1552\, \mu mol \cdot a)$$

If we set NPV = 0,
$$ -374518725\, dollars + \sum_{t=2}^{5}(1.041)^{1-t}(52871.1552\, \mu mol \cdot a) = 0 $$
$$\sum_{t=2}^{5}(1.041)^{1-t}(52871.1552\, \mu mol \cdot a) = 374518725\, dollars $$
$$\sum_{t=2}^{5}(1.041)^{1-t}(a) = 7083.61154 {dollars\over \mu mol}$$
$$((1.041)^{-1} \cdot a)+((1.041)^{-2} \cdot a)+((1.041)^{-3} \cdot a)+((1.041)^{-4} \cdot a) = 7083.61154 {dollars\over \mu mol}$$
$$a((1.041)^{-1}+(1.041)^{-2}+(1.041)^{-3}+(1.041)^{-4}) = 7083.61154 {dollars\over \mu mol} $$
$$a \approx 1956.07 {dollars\over \mu mol} $$

$\quad$

The price for a micromole of mRNA is $1956.07 {dollars\over \mu mol}$. Recall that this value is only if the mRNA that is produced by this process is sold, and all the unreacted starting materials are disposed. It is obvious that the high value of $CF_1$ can be attributed to the purchase of PURExpress. It is obvious then that reducing the amount of PURExpress that is consumed and subsequently wasted will boost the economic feasibility of this design. 

This means that under this design process, each micromole of mRNA-BNT162b2 would have to be sold at a retail price of ***1956.07 dollars*** to generate the same income as an alternative investment yieldign 4.1 percent per year. 

Here are a few measures that can be taken to do so: 
* The PURExpress that flows out of node 3 can be recycled via a stream to reactor chip 1. However, it is important to mention that the PURExpress that flows out of node 3 has a temperature of -25℃ because of the heat exchanger stationed behind the MSU. For this reason, another heat exchanger will be required to heat the PURExpress solution to 37℃
* Like Pfizer and many other pharmaceutical companies, we could produce our version of the PURExpress solution. This would greatly reduce the costs.
* Additionally, we could strike a deal with the manufacturers of the PURExpress in-vitro expression kits in order to attain PURExpress at a cheaper price.

## Summary and conclusions
* Fill me in, see rubric.

This project utilized 2 syringe pumps, 3 heat exchangers, 15 reactor chips, and 2 storage vessels. In doing so, the total expenses for a duration of 5 years came down to 347,518,725 dollars. While we were able to design a process that meets the technical specifications of 0.1 micro moles of 99% purity mRNA every minute, there are still multiple ways we can make this design more economically feasible. One way of doing so is by recycling the excess PUREExpress left at the end of the reaction system. Assuming there was 20% left at the end, we could expend 20% less money on PURExpress by reinserting that excess material back into the system. This would make it so we could use an initial value of PURExpress 80% of the original value, drastically decreasing our expenses. This being said, the assumptions would be that the PURExpress still functions the same as the initial concentration. Another way to decrease our expenses would be through investing in the research of chip process because in using more chips the cost increased directly proportional to number of chips. And considering that initial cost is negligible, it would be of great value to decrease the amount of PURExpress needed. Of indefinite advantage would be to produce our own PURExpress. In doing so, the majority of the expense would be reduced down to however much is necessary to produce the PURExpress. All of these things could dramatically impact cost and are of great benefit.

## References and Additional Resources
* “Shell and Tube Heat Exchanger Fouling - Klaren International BV.” Klaren International - Just another WordPress site. Klaren International, October 16, 2022. https://klarenbv.com/heat-exchanger-basics/. 
* Biolabs, New England. “PURExpress® in Vitro Protein Synthesis Kit.” PURExpress In-Vitro Expression Kit. New England Biolabs. Accessed December 15, 2022. https://www.neb.com/products/e6800-purexpress-invitro-protein-synthesis-kit#Product%20Information_Kit%20Components. 
* Khehra, Nimrat, Inderbir Padda, Urooj Jaferi, Harshan Atwal, Shreya Narain, and Mayur S Parmar. “Tozinameran (BNT162B2) Vaccine: The Journey from Preclinical Research to Clinical Trials and Authorization.” AAPS PharmSciTech. U.S. National Library of Medicine, June 7, 2021. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8184133/. 
* Gao, Weiwei, Che-Ming  J Hu, Ronnie  H Fang, and Liangfang Zhang. “Liposome-like Nanostructures for Drug Delivery.” Journal of materials chemistry. B. U.S. National Library of Medicine, December 28, 2013. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3877745/. 
* Polack, Fernando P, Stephen J Thomas, Nicholas Kitchin, Judith Absalon, Alejandra Gurtman, Stephen Lockhart, John L Perez, et al. “Safety and Efficacy of the BNT162B2 Mrna Covid-19 Vaccine.” The New England journal of medicine. U.S. National Library of Medicine, December 31, 2020. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7745181/. 
* CDC. “Grade: Moderna Covid-19 Vaccine.” Centers for Disease Control and Prevention. Centers for Disease Control and Prevention, December 20, 2020. https://www.cdc.gov/vaccines/acip/recs/grade/covid-19-moderna-vaccine.html. 
* Orth, Jeffrey  D, Ines Thiele, and Bernhard Ø Palsson. “What Is Flux Balance Analysis?” Nature biotechnology. U.S. National Library of Medicine, June 6, 2011. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3108565/. 
* ENGRI-1120 Textbook, Intro to Flux Balance Analysis and Metabolic Engineering.